# 简介

这是一款名为‘FunctionAgentWithRetrieval检索增强小助手’的演示应用，展示了如何结合[llama_index](https://github.com/run-llama/llama_index.git)的工具和ERNIEBot的function agent来回答用户的专业知识问题。首先，应用通过构建FaissSearchLlamaIndex在知识库中检索相关内容，然后评估检索内容与问题的相关度。如果内容与问题高度相关，大模型将采用检索增强方式回答问题；否则，大模型将调用工具列表中的工具回答用户的问题。这种设计不仅扩展了大模型的专业领域知识，还保持了其在领域知识之外的通用对话能力

构建流程如下：

# 1. 导入第三方库

主要是在导入一些必要的Python库和模块，以便实现FunctionAgentWithRetrival的功能。
+ os: Python的标准库，用于与操作系统进行交互，如读写文件、管理路径等。
+ faiss: 用于向量存储的模块，是用于存储和检索经过嵌入处理的文本或图像的向量表示。
+ SimpleDirectoryReader: 用于在文件夹中加载数据
+ ErnieEmbeddings: 用于文本嵌入的工具，将文本转换为可以在模型中使用的向量表示。
+ FunctionAgent: 这个类实现function calling功能的Agent的类，如问答、对话等。
+ WholeMemory: 用于存储和管理代理的记忆的类。
+ ERNIEBot: 实现ERNIE Bot的主要类，包含了实现对话功能的主要逻辑。
+ erniebot: 这是导入erniebot模块的语句，erniebot是一个包含ERNIE Bot实现的主要代码库。
+ load_index_from_storage: 用于从存储系统加载索引。
+ VectorStoreIndex: 提供了创建索引、添加数据到索引等功能。
+ StorageContext: 用于存储节点的实用容器。
+ FaissVectorStore: 提供了一种使用 FAISS 库来存储和检索向量的方法。
+ ServiceContext: 服务上下文容器是 LlamaIndex 的实用程序容器。

这里只调用llama_index的检索功能，os.environ["OPENAI_API_KEY"]可以赋值空字符

这里使用EB_AGENT_ACCESS_TOKEN, 申请地址请参考[accessToken](https://aistudio.baidu.com/index/accessToken)


In [1]:
import os

os.environ["OPENAI_API_KEY"] = " "
os.environ["EB_AGENT_ACCESS_TOKEN"] = "your access token"

from erniebot_agent.agents.function_agent_with_retrieval import FunctionAgentWithRetrieval
from erniebot_agent.memory.whole_memory import WholeMemory
from erniebot_agent.chat_models.erniebot import ERNIEBot
from erniebot_agent.tools import RemoteToolkit
from erniebot_agent.extensions.langchain.embeddings import ErnieEmbeddings
import faiss
from llama_index import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
    ServiceContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore

# 2. 预处理

## 2.1 下载数据集

In [4]:
! wget https://paddlenlp.bj.bcebos.com/datasets/examples/construction_regulations.tar
! tar xvf construction_regulations.tar

--2023-12-26 02:53:33--  https://paddlenlp.bj.bcebos.com/datasets/examples/construction_regulations.tar
Resolving paddlenlp.bj.bcebos.com (paddlenlp.bj.bcebos.com)... 36.110.192.178, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddlenlp.bj.bcebos.com (paddlenlp.bj.bcebos.com)|36.110.192.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1239040 (1.2M) [application/x-tar]
Saving to: ‘construction_regulations.tar’

construction_regula 100%[===================>]   1.18M  4.98MB/s    in 0.2s    

2023-12-26 02:53:33 (4.98 MB/s) - ‘construction_regulations.tar’ saved [1239040/1239040]

construction_regulations/
construction_regulations/城市管理执法办法.pdf
construction_regulations/建筑工程设计招标投标管理办法.pdf
construction_regulations/建筑业企业资质管理规定.pdf
construction_regulations/城市照明管理规定.pdf
construction_regulations/城市设计管理办法.pdf
construction_regulations/建筑工程施工发包与承包计价管理办法.pdf
construction_regulations/市政公用设施抗灾设防管理规定.pdf


## 2.2 构建FaissSearch_lama
这段代码定义了一个名为 FaissSearchLlamaIndex 的类，它使用 FAISS 库进行文档搜索。这个类使用 FAISS 来创建一个基于文档的向量存储索引，并提供了一个搜索方法来查询索引并返回最相关的文档。这种方法适用于需要快速且高效地处理大量高维向量数据的场景，如文本相似性搜索。

In [2]:
class FaissSearchLlamaIndex:
    def __init__(
        self,
        faiss_index,
        documents,
        embed_model,
    ):
        # 类的初始化方法，接收 FAISS 索引、文档集合和嵌入模型作为参数
        self.vector_store = FaissVectorStore(faiss_index=faiss_index)
        # 初始化 FaissVectorStore，用于存储和检索向量
        storage_context = StorageContext.from_defaults(vector_store=self.vector_store)
        # 创建一个storage_context，配置默认值和向量存储
        self.service_context = ServiceContext.from_defaults(embed_model=embed_model)
        # 创建一个service_context，配置默认值和嵌入模型
        self.index = VectorStoreIndex.from_documents(
            documents,
            storage_context=storage_context,
            show_progress=True,
            service_context=self.service_context,
        )
        # 从文档集合创建一个向量存储索引，并显示进度
        self.index.storage_context.persist()
        # 持久化索引数据到存储

    def search(self, query, topk=10, **kwargs):
        # 定义搜索方法，接收查询字符串、返回结果数量上限以及其他参数
        storage_context = StorageContext.from_defaults(
            vector_store=self.vector_store, persist_dir="./storage"
        )
        # 创建一个新的存储上下文，指定持久化目录
        index = load_index_from_storage(
            storage_context=storage_context, service_context=self.service_context, top_k=topk
        )
        # 从存储中加载索引
        retriever = index.as_retriever()
        # 从索引中创建检索器
        nodes = retriever.retrieve(query)
        # 使用检索器对查询进行检索
        retrieval_results = []
        for doc in nodes:
            # 遍历检索到的文档
            retrieval_results.append(
                {"content": doc.node.text, "score": doc.score, "title": doc.node.metadata["file_name"]}
            )
            # 将文档的内容、评分和标题添加到结果列表中
        return retrieval_results  # 返回搜索结果

# 3.FunctionAgentWithRetrieval
## 3.1 建索引库

这段代码主要功能是用于创建或加载一个FAISS索引来进行文档相似度匹配。首先，创建一个ErnieEmbeddings对象，用于生成文档的嵌入向量。aistudio_access_token是访问AI Studio的令牌，chunk_size是用于嵌入的文档块的数量

In [3]:
aistudio_access_token = os.environ.get("EB_AGENT_ACCESS_TOKEN", "")
embed_model = ErnieEmbeddings(aistudio_access_token=aistudio_access_token, chunk_size=16)

接下来利用ErnieEmbeddings来抽取向量构建索引。d是ernie-embedding的向量抽取维度,并向SimpleDirectoryReader传入数据存储的目录

In [4]:
d = 384
faiss_index = faiss.IndexFlatIP(d)
documents = SimpleDirectoryReader("construction_regulations").load_data()

以下代码是使用FaissSearchLlamaIndex进行搜索的一个例子，流程大致如下：
1. 创建FaissSearchLlamaIndex对象，传入embed_model、documents、faiss_index。
2. 调用FaissSearchLlamaIndex对象的search函数，并传入查询字符串"城市管理执法主管部门的职责是什么？"。
3. 将搜索结果以格式化的形式进行打印，这里使用了pprint模块进行美化打印。最终的结果存储在变量res中。

In [5]:
faiss_search = FaissSearchLlamaIndex(embed_model=embed_model, documents=documents, faiss_index=faiss_index)
query = "城市管理执法主管部门的职责是什么？"
res = faiss_search.search(query)
from pprint import pprint

pprint(res)

/root/qingzhong/anas/envs/py310_openai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|█████████████████████████████████████████████████████████| 82/82 [00:47<00:00,  1.73it/s]


[{'content': '住房和城乡建设部规章 \n'
             '   X住房和城乡建设部发布  \n'
             '- 11 -   第三十八条 城市管理执法主管部门应当向社会公布投诉、举报电话及其他监督方式。   '
             '城市管理执法主管部门应当为投诉人、举报人保密。   第三十九条 城市管理执法主管部门违反本办法规定， 有下列行为之一的， '
             '由上级城市管理执法主管部门或者有关部门责令改正，通报批评；情节严重的，对直接负责的主管人员和其他直接责任人员依法给予处分。   '
             '（一）没有法定依据实施行政处罚的；   （二）违反法定程序实施行政处罚的；   （三）以罚款、没收违法所得作为经费来源的；   '
             '（四）使用、截留、损毁或者擅自处置查封、扣押物品的；   （五）其他违反法律法规和本办法规定的。   第四十条 '
             '非城市管理执法人员着城市管理执法制式服装的，城市管理执法主管部门应当予以纠正，依法追究法律责任。',
  'score': 0.5666415095329285,
  'title': '城市管理执法办法.pdf'},
 {'content': '住房和城乡建设部规章 \n'
             '   X住房和城乡建设部发布  \n'
             '- 10 -   第三十四条 城市管理执法主管部门应当通过门户网站、 '
             '办事窗口等渠道或者场所，公开行政执法职责、权限、依据、监督方式等行政执法信息。   第六章 协作与配合     第三十五条 '
             '城市管理执法主管部门应当与有关部门建立行政执法信息互通共享机制， 及时通报行政执法信息和相关行政管理信息。   第三十六条 '
             '城市管理执法主管部门可以对城市管理执法事项实行网格化管理。   第三十七条 '
             '城市管理执法主管部门在执法活动中发现依法应当由其他部门查处的违法行为， 应当及时告知或者移送有关部门。   第七章 执法监督',
  'score': 0.5620024800300598,
  't

## 3.2 构建FunctionAgentWithRetrieval
FunctionAgentWithRetrieval可以决定是否进行检索增强来回答用户问题，以下是其示例。
### 3.2.1 检索增强

In [9]:
# 创建一个ERNIEBot实例，使用"ernie-3.5"模型。
llm = ERNIEBot(model="ernie-3.5")
# 创建一个WholeMemory实例。这是一个用于存储对话历史和上下文信息的类，有助于模型理解和持续对话。
memory = WholeMemory()
# 调用文本语音转换工具。
tts_tool = RemoteToolkit.from_aistudio("texttospeech").get_tools()[0]
# 创建一个FunctionAgentWithRetrieval实例。这个代理将使用上面创建的ERNIEBot模型、WholeMemory和faiss_search，同时传入了一个名为tts_tool的工具。
agent = FunctionAgentWithRetrieval(
    llm=llm, tools=[tts_tool], memory=memory, knowledge_base=faiss_search, threshold=0.5
)

In [10]:
# 定义一个查询字符串，这个查询是关于"城乡建设部规章中，城市管理执法第三章，第十三条"的内容。
query = "城乡建设部规章中，城市管理执法第三章，第十三条是什么？"
# 使用agent的async_run方法来异步执行查询。由于这是异步操作，因此需要使用'await'关键字。
response = await agent._run(query)
messages = response.chat_history
for item in messages:
    print(item.to_dict())

{'role': 'user', 'content': '检索结果:\n\n    第1个段落: 住房和城乡建设部规章 \n   X住房和城乡建设部发布  \n- 9 -   第三十条 城市管理执法主管部门不得对罚款、 没收违法所得设定任务和目标。   罚款、没收违法所得的款项，应当按照规定全额上缴。   第三十一条 城市管理执法主管部门应当确定法制审核机构，配备一定比例符合条件的法制审核人员，对重大执法决定在执法主体、管辖权限、执法程序、事实认定、法律适用等方面进行法制审核。   第三十二条 城市管理执法主管部门开展执法活动， 应当使用统一格式的行政执法文书。   第三十三条 行政执法文书的送达， 依照民事诉讼法等法律规定执行。   当事人提供送达地址或者同意电子送达的， 可以按照其提供的地址或者传真、电子邮件送达。   采取直接、留置、邮寄、委托、转交等方式无法送达的，可以通过报纸、门户网站等方式公告送达。\n\n    第2个段落: 住房和城乡建设部规章 \n   X住房和城乡建设部发布  \n- 4 -   第十一条 城市管理执法事项范围确定后，应当向社会公开。   第十二条 城市管理执法主管部门集中行使原由其他部门行使的行政处罚权的，应当与其他部门明确职责权限和工作机制。   第三章 执法主体     第十三条 城市管理执法主管部门按照权责清晰、事权统一、精简效能的原则设置执法队伍。   第十四条 直辖市、 设区的市城市管理执法推行市级执法或者区级执法。   直辖市、设区的市的城市管理执法事项，市辖区人民政府城市管理执法主管部门能够承担的，可以实行区级执法。   直辖市、 设区的市人民政府城市管理执法主管部门可以承担跨区域和重大复杂违法案件的查处。\n\n检索语句: 城乡建设部规章中，城市管理执法第三章，第十三条是什么？\n请根据以上检索结果回答检索语句的问题'}
{'role': 'assistant', 'content': '根据提供的检索结果，城乡建设部规章中，城市管理执法第三章，第十三条的内容是：“城市管理执法主管部门按照权责清晰、事权统一、精简效能的原则设置执法队伍。”', 'function_call': None, 'plugin_info': None, 'search_info': {'results': [{'index': 1, '

### 3.2.2 调用tool示例
在这个例子中，FunctionAgentWithRetrieval没有利用其检索信息，而是直接调用工具回答用户的问题

In [11]:
# 定义一个查询字符串，这个查询是关于"把上一轮的检索内容转换为语音"的内容。
response = await agent._run("把上一轮的检索内容转换为语音")
# 使用agent的async_run方法来异步执行查询。由于这是异步操作，因此需要使用'await'关键字。
messages = response.chat_history
for item in messages:
    print(item.to_dict())

{'role': 'user', 'content': '把上一轮的检索内容转换为语音'}
{'role': 'assistant', 'content': '', 'function_call': {'name': 'texttospeech/v1.6/tts', 'thoughts': '用户需要将检索内容转换为语音。', 'arguments': '{"tex":"城市管理执法主管部门按照权责清晰、事权统一、精简效能的原则设置执法队伍。"}'}, 'plugin_info': None, 'search_info': None}
{'role': 'function', 'name': 'texttospeech/v1.6/tts', 'content': '{"audio": "file-local-095d5f98-a480-11ee-af84-fa280016e4f3", "prompt": "参考工具说明中对各个结果字段的描述，提取工具调用结果中的信息，生成一段通顺的文本满足用户的需求。请务必确保每个符合\'file-\'格式的字段只出现一次，无需将其转换为链接，也无需添加任何HTML、Markdown或其他格式化元素。"}'}
{'role': 'assistant', 'content': '根据您提供的文本，我为您生成了语音文件。请注意，由于技术限制，语音文件无法直接提供给您。您可以通过访问以下链接来获取语音文件：file-local-095d5f98-a480-11ee-af84-fa280016e4f3。如果您需要进一步操作或有其他问题，请随时告诉我。', 'function_call': None, 'plugin_info': None, 'search_info': None}
